In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## import

In [51]:
import numpy as np
import pandas as pd
from kite_trade import *
import os
# import ta

import talib
import datetime
import pyarrow.parquet as pq
import pyarrow as pa

import multiprocessing
import time
from tqdm import tqdm
import util_fun as uf

from pandas.tseries.offsets import MonthEnd
from datetime import timedelta

import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
import mplfinance as mpf


from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA

from kite_trade import *
from enctoken import get_kite
kite = get_kite()

In [52]:
timerframe_list = [
"day",
"minute",
 "3minute",
 "5minute",
 "10minute",
 "15minute",
 "30minute",
 "60minute",]

# instument in nse
inst = pd.DataFrame(kite.instruments("NSE"))
inst_filter = inst.query('(name != "")').copy()
inst_filter.rename(columns = {"tradingsymbol":'Symbol'},inplace = True)
# inst_filter.query("Symbol. 'NIFTY'")
inst_filter[inst_filter.Symbol.str.contains('HDFCBANK')]


,instrument_token,exchange_token,Symbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
3180,341249,1333,HDFCBANK,HDFC BANK,0.0,None,0.0,0.05,1,EQ,NSE,NSE


In [8]:

# nifty_500 = pd.read_csv('../data/ind_nifty500list.csv')
# nifty_500 = nifty_500.merge(inst_filter, on = 'Symbol')
# inst_dict = dict(zip(nifty_500.Symbol, nifty_500.instrument_token))

# start_dt = '2018-01-01'
# end_dt = '2023-02-02'
# time_frame = 'day'
# for symbol, instument in tqdm(inst_dict.items()):
#     print(symbol, instument)
#     df_day = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
#     directory = f'../data/historical/{time_frame}/{symbol}'
#     if not os.path.exists(directory):
#         os.makedirs(directory)
#     df_day.to_parquet(f'{directory}/part0.parquet')
#     # break
          

In [9]:
df_day = df_day.between_time('09:15', '03:25')
df_day = df_day[df_day.Volume != 0]
df_day['ema_5'] = talib.EMA(df_day['Close'], timeperiod = 5)
df_day.dropna(inplace=True)
df_day['above_5_ema'] = df_day['Low'] > df_day['ema_5']


NameError: name 'df_day' is not defined

In [277]:
# df_day = df_day[df_day.index.strftime('%Y-%m-%d').isin(["2023-01-18","2023-01-19"]) ].copy()

## Backtesting 

## 5 ema

In [278]:
# 5 ema above is my signal candle
# if  next candle is signal  dont do anything
# if next candle is 

# if current one is crossing previous candle low and previous candle is above 5 ema

In [41]:
start_dt = '2023-01-01'
end_dt = '2023-04-01'
time_frame = '5minute'
instument = 256265 #256265(nifty 260105
df_5min = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
df_day_fetch = uf.get_data_parllel(kite, instument, 'day' , start_dt,end_dt)
temp1 = df_day_fetch.copy()
temp2 = df_5min.copy()

Pandas Apply:   0%|          | 0/3 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/3 [00:00<?, ?it/s]

In [42]:
df_5min = temp2.copy()
df_5min = df_5min.between_time('09:15', '15:20').copy()
df_5min.index = df_5min.index.tz_localize(None)

In [43]:
def prev_ret(data):
    # print(data)
    return data

In [44]:
class RsiOscilator(Strategy):
    back = 5
    rtr = 2
    flag = 0
    date_list = []
    std_dev = 1.5
    signal_candle_low = 0
    signal_candle_high = 0
    start_candle  = '09:15:00'
    risk_reward_ratio = .98
    def init(self):
        self.ema_5 = self.I(talib.EMA, self.data.Close, self.back)
        self.previous_low = self.I(prev_ret,self.data.Low,plot=False)
        self.previous_high = self.I(prev_ret,self.data.High,plot=False)
    
    def next(self):
        # print(self.ema_5)
        # print(self.data.Close)
        if (self.data.index.date[-1] not in self.date_list) and (self.data.index[-1].time() == pd.to_datetime(self.start_candle).time()):
            self.flag = 3
            self.signal_candle_high = 0
            self.signal_candle_low = 0
            self.date_list.append(self.data.index.date[-1])
        elif self.data.index[-1].time() > pd.to_datetime('15:15:00').time():
            self.position.close()

        elif  (self.previous_low[-2] > self.ema_5[-2]) and (self.data.Close[-1] < self.previous_low[-2]):
            stop_loss = max(self.data.High[-1:-4:-1])
            target = (stop_loss - self.data.Close[-1]) * self.rtr
            # print(f'stop_loss, target', stop_loss,self.data.Close[-1] - target)
            self.sell(sl =stop_loss, tp =self.data.Close[-1] - target  ) 
        # elif self.data.High[-1]< self.ema_5:    
        #     self.position.close() 



       
        # if (self.data.index.date[-1] not in self.date_list) and (self.data.index[-1].time() == pd.to_datetime(self.start_candle).time()):
        #     self.flag = 3
        #     self.signal_candle_high = 0
        #     self.signal_candle_low = 0
        #     self.date_list.append(self.data.index.date[-1])

        # elif self.data.index[-1].time() > pd.to_datetime('15:15:00').time():
        #     self.position.close()

        # elif self.data.Low[-1] > self.upper[-1]:
        #     self.signal_candle_high = self.data.High[-1]
        #     self.signal_candle_low = self.data.Low[-1]

        # elif self.data.Close[-1] < self.signal_candle_low and  self.signal_candle_low != 0 and  self.signal_candle_high != 0 and self.flag > 0:
        #     self.sell(sl = self.signal_candle_high, tp = self.data.Close[-1] * self.risk_reward_ratio)
        #     self.flag -= 1
        #     self.signal_candle_high = 0
        #     self.signal_candle_low = 0

bt = Backtest(df_5min,RsiOscilator, cash = 100000)

In [49]:
stats = bt.run()
# stats = bt.optimize(
#     # back = range(3,10,1),
#     rtr = range(1,3,1)  ,
#     maximize=  'Return [%]'
# )
stats

Start                     2023-01-02 09:15:00
End                       2023-03-10 15:20:00
Duration                     67 days 06:05:00
Exposure Time [%]                   34.543919
Equity Final [$]                     98792.75
Equity Peak [$]                      100762.5
Return [%]                           -1.20725
Buy & Hold Return [%]               -3.812434
Return (Ann.) [%]                   -6.257579
Volatility (Ann.) [%]                5.825581
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -3.553902
Avg. Drawdown [%]                   -0.596432
Max. Drawdown Duration       39 days 06:05:00
Avg. Drawdown Duration        5 days 20:18:00
# Trades                                  116
Win Rate [%]                        36.206897
Best Trade [%]                       0.877542
Worst Trade [%]                     -1.415209
Avg. Trade [%]                    

In [47]:
stats._strategy

<Strategy RsiOscilator(rtr=2)>

In [39]:
bt.plot()

Row(id='2532', ...)

## vwape

In [286]:
after 10.45:
	if price below vwap:
		Srsi below is below red 
		it it breaks the next below pivot 
			Sell -> stop loss above vwap and target 1%
	if price above vwap:
		srsi above the red
		if it breaks the next above pivot :
			buy stop loss above vwap and target 1%
  

SyntaxError: invalid syntax (3291927124.py, line 1)

In [ ]:
start_date = '2023-01-01'
end_data = '2023-01-20'
symbol = 'HDFCBANK'
df_day = uf.get_instrument_data(
            kite,
            inst_filter,
            timerframe_list = timerframe_list[3:4],
            start_date =start_date ,
            end_date = end_data,
            symbol = symbol)
            

df_day = df_day.between_time('10:45', '03:25')
df_day = df_day[df_day.Volume != 0]
# df_day['ema_5'] = talib.EMA(df_day['Close'], timeperiod = 5)
# df_day.dropna(inplace=True)
# df_day['above_5_ema'] = df_day['Low'] > df_day['ema_5']


2023-01-01 2023-01-20
5minute


100%|██████████| 1/1 [00:01<00:00,  1.85s/it]


In [ ]:
import ta

In [ ]:
# ta.volume.VolumeWeightedAveragePrice(df_day['High'], df_day['Low'], df_day['Close'], df_day['Volume'])

## nifty future intraday streategy 3rd candle

In [500]:
start_dt = '2020-05-01'
end_dt = '2023-02-02'
time_frame = '5minute'
instument = 256265
df_5min = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
df_day_fetch = uf.get_data_parllel(kite, instument, 'day' , start_dt,end_dt)
temp1 = df_day_fetch.copy()
temp2 = df_5min.copy()

Pandas Apply:   0%|          | 0/33 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/33 [00:00<?, ?it/s]

In [501]:
df_day_fetch.head()

,Open,High,Low,Close,Volume
Date,,,,,
2020-05-04 00:00:00+05:30,9533.50,9533.50,9266.95,9293.50,0
2020-05-05 00:00:00+05:30,9429.40,9450.90,9190.75,9205.60,0
2020-05-06 00:00:00+05:30,9226.79,9346.90,9116.50,9270.90,0
2020-05-07 00:00:00+05:30,9234.04,9277.85,9175.90,9199.04,0
2020-05-08 00:00:00+05:30,9376.95,9382.65,9238.20,9251.50,0


In [524]:
def get_day_pivot_added(df_day_fetch, df_min_5):
    df_day_fetch_temp = df_day_fetch.copy()
    
    #calculating pivot
    df_day_fetch['pivot'] = (df_day_fetch['High'] + df_day_fetch['Low'] + df_day_fetch['Close'])/3
    df_day_fetch['bottom_pivot'] =  (df_day_fetch['High'] + df_day_fetch['Low'])/2
    df_day_fetch['top_pivot'] = (df_day_fetch['pivot'] - df_day_fetch['bottom_pivot']) + df_day_fetch['pivot']
    df_day_fetch = df_day_fetch.shift(1)

    # adding preiouse day price 
    df_day_fetch.rename(
        columns = {'Close': "prev_close", 'High':"prev_high","Low": "prev_low",
        "Open":'prev_open'}, inplace = True)
    df_day_fetch = df_day_fetch_temp.merge(df_day_fetch,left_index=True, right_index=True)

    # merging pivot with 5 min candle
    df_min_5['date_only'] = pd.to_datetime(df_min_5.index.date)
    df_day_fetch['date_only'] = pd.to_datetime(df_day_fetch.index.date)
    df_day_fetch.rename(
        columns = {'Close': "day_close", 'High':"day_high","Low": "day_low",
        "Open":'day_open'}, inplace = True)
    df_min_5.reset_index(inplace = True)
    df_min_5 = df_min_5.merge(df_day_fetch,how = 'left', on = 'date_only')
    df_min_5.set_index('Date',inplace = True)
    df_min_5.dropna(inplace =True)
    df_min_5.drop(['Volume_x','Volume_y'],inplace = True,axis =1 )
    
    return df_min_5


In [525]:
df_day_fetch = temp1.copy()
df_5min = temp2.copy()
df_5min = get_day_pivot_added(df_day_fetch, df_5min)


In [526]:
df_5min.head()

,Open,High,Low,Close,Volume,date_only,day_open,day_high,day_low,day_close,prev_open,prev_high,prev_low,prev_close,pivot,bottom_pivot,top_pivot
Date,,,,,,,,,,,,,,,,,
2020-05-05 09:15:00+05:30,9429.40,9450.00,9415.00,9441.55,0,2020-05-05,9429.4,9450.9,9190.75,9205.6,9533.5,9533.5,9266.95,9293.5,9364.65,9400.225,9329.075
2020-05-05 09:20:00+05:30,9441.35,9448.70,9416.40,9417.15,0,2020-05-05,9429.4,9450.9,9190.75,9205.6,9533.5,9533.5,9266.95,9293.5,9364.65,9400.225,9329.075
2020-05-05 09:25:00+05:30,9416.75,9416.75,9388.20,9391.60,0,2020-05-05,9429.4,9450.9,9190.75,9205.6,9533.5,9533.5,9266.95,9293.5,9364.65,9400.225,9329.075
2020-05-05 09:30:00+05:30,9391.60,9411.80,9380.00,9399.40,0,2020-05-05,9429.4,9450.9,9190.75,9205.6,9533.5,9533.5,9266.95,9293.5,9364.65,9400.225,9329.075
2020-05-05 09:35:00+05:30,9399.75,9422.05,9397.15,9418.00,0,2020-05-05,9429.4,9450.9,9190.75,9205.6,9533.5,9533.5,9266.95,9293.5,9364.65,9400.225,9329.075


In [527]:
df_5min['close_1per'] = df_5min['prev_close'] * .001
df_5min['tc-bc'] = abs(df_5min['top_pivot'] - df_5min['bottom_pivot'])
df_5min['cpr_range']  = np.where(df_5min['tc-bc'] < df_5min['close_1per'], "narrow", 'wide')

In [528]:
df_5min = df_5min.between_time('09:15', '15:10').copy()
# df_5min.loc['date_only'] = df_5min.index.date
df_5min['low_down'] = np.where(df_5min.index.time == pd.to_datetime('09:25:00').time(),df_5min['Low'] - 5, np.nan)
df_5min['high_up'] = np.where(df_5min.index.time == pd.to_datetime('09:25:00').time(),df_5min['High'] - 5, np.nan)
df_5min.ffill(inplace=True)
df_5min.index = df_5min.index.tz_localize(None)
df_5min.dropna(inplace = True)

In [529]:
df_5min.tail()

,Open,High,Low,Close,Volume,date_only,day_open,day_high,day_low,day_close,...,prev_low,prev_close,pivot,bottom_pivot,top_pivot,close_1per,tc-bc,cpr_range,low_down,high_up
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-20 14:50:00,18042.05,18056.00,18041.55,18047.55,0,2023-01-20,18115.6,18145.45,18016.2,18027.65,...,18063.75,18107.85,18108.933333,18109.475,18108.391667,18.10785,1.083333,narrow,18071.35,18103.25
2023-01-20 14:55:00,18046.75,18049.50,18038.85,18049.40,0,2023-01-20,18115.6,18145.45,18016.2,18027.65,...,18063.75,18107.85,18108.933333,18109.475,18108.391667,18.10785,1.083333,narrow,18071.35,18103.25
2023-01-20 15:00:00,18050.40,18054.25,18032.90,18035.25,0,2023-01-20,18115.6,18145.45,18016.2,18027.65,...,18063.75,18107.85,18108.933333,18109.475,18108.391667,18.10785,1.083333,narrow,18071.35,18103.25
2023-01-20 15:05:00,18035.30,18037.70,18020.05,18030.95,0,2023-01-20,18115.6,18145.45,18016.2,18027.65,...,18063.75,18107.85,18108.933333,18109.475,18108.391667,18.10785,1.083333,narrow,18071.35,18103.25
2023-01-20 15:10:00,18031.15,18032.90,18024.75,18029.35,0,2023-01-20,18115.6,18145.45,18016.2,18027.65,...,18063.75,18107.85,18108.933333,18109.475,18108.391667,18.10785,1.083333,narrow,18071.35,18103.25


In [533]:
def low_high(data):
    # print(data)
    return data

class nifty_3candle(Strategy): 
    flag = 0
    date_list = []
    def init(self):
        self.low_down = self.I(low_high, self.data.low_down, plot = False)
        self.high_up = self.I(low_high, self.data.high_up , plot = False)
    
    def next(self):
        # print(self.data.index.date)
        if (self.data.index.date[-1] not in self.date_list) and (self.data.index[-1].time() == pd.to_datetime('09:25:00').time()):
            self.flag = 0
            self.date_list.append(self.data.index.date[-1])
        

        if self.data.index[-1].time() == pd.to_datetime('15:05:00').time():
            self.position.close()
        
        
        if self.data.index[-1].time() > pd.to_datetime('09:25:00').time():
            if self.data.Close[-1] < self.low_down[-1] and not self.position and self.flag == 0 and self.data.cpr_range[-1] == 'wide':
                self.sell(sl = self.high_up[-1])
                self.flag = 1
            elif self.data.Close[-1] > self.high_up[-1] and not self.position and self.flag == 0  and self.data.cpr_range[-1] == 'wide':
                self.buy(sl = self.low_down[-1])
                self.flag = 1

bt = Backtest(df_5min,nifty_3candle, cash = 100000)

In [534]:
stats = bt.run()
stats

Start                     2020-05-05 09:25:00
End                       2023-01-20 15:10:00
Duration                    990 days 05:45:00
Exposure Time [%]                   36.945703
Equity Final [$]                     120776.8
Equity Peak [$]                      121047.7
Return [%]                            20.7768
Buy & Hold Return [%]               91.973146
Return (Ann.) [%]                    7.413989
Volatility (Ann.) [%]                7.396464
Sharpe Ratio                         1.002369
Sortino Ratio                        2.538889
Calmar Ratio                         1.413355
Max. Drawdown [%]                   -5.245665
Avg. Drawdown [%]                   -0.375521
Max. Drawdown Duration      269 days 21:50:00
Avg. Drawdown Duration        5 days 14:37:00
# Trades                                  513
Win Rate [%]                        29.239766
Best Trade [%]                       2.679942
Worst Trade [%]                     -1.048326
Avg. Trade [%]                    

In [475]:
# stats._equity_curve.plot()

In [532]:
bt.plot(resample = False)

Row(id='21891', ...)

In [510]:
df_trades = stats._trades

In [523]:
flag_addup = 0
cons_list = []
prev_val =df_trades['PnL'][0]
for i in df_trades['PnL']:
    if prev_val > 0 and i < 0:
        flag_addup += 1
        cons_list.append(flag_addup)
    elif prev_val < 0 and i > 0:
        flag_addup += 1
        cons_list.append(flag_addup)
    else:
        cons_list.append(flag_addup)
    prev_val = i 
df_trades['flag_addup'] = cons_list
df_trades['entry_date'] = df_trades['EntryTime'].dt.date
        

In [522]:
df_trades[(df_trades['entry_date'] > pd.to_datetime("2020-09-01")) & (df_trades['entry_date'] < pd.to_datetime("2021-02-27"))]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,flag_addup,year,month,day_name,entry_date
67,8,6843,6865,11557.75,11541.90,-126.8,-0.001371,2020-09-03 09:40:00,2020-09-03 11:30:00,0 days 01:50:00,24,2020,9,Thursday,2020-09-03
68,8,6915,6975,11371.75,11345.90,-206.8,-0.002273,2020-09-04 09:40:00,2020-09-04 14:40:00,0 days 05:00:00,24,2020,9,Friday,2020-09-04
69,-8,6986,7007,11309.45,11338.70,-234.0,-0.002586,2020-09-07 09:35:00,2020-09-07 11:20:00,0 days 01:45:00,24,2020,9,Monday,2020-09-07
70,8,7059,7122,11359.95,11338.45,-172.0,-0.001893,2020-09-08 09:40:00,2020-09-08 14:55:00,0 days 05:15:00,24,2020,9,Tuesday,2020-09-08
71,8,7130,7133,11266.75,11244.85,-175.2,-0.001944,2020-09-09 09:35:00,2020-09-09 09:50:00,0 days 00:15:00,24,2020,9,Wednesday,2020-09-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,6,16059,16105,15058.20,15025.75,-194.7,-0.002155,2021-02-19 09:40:00,2021-02-19 13:30:00,0 days 03:50:00,62,2021,2,Friday,2021-02-19
156,-6,16135,16197,14926.80,14699.00,1366.8,0.015261,2021-02-22 10:00:00,2021-02-22 15:10:00,0 days 05:10:00,63,2021,2,Monday,2021-02-22
157,-6,16202,16210,14733.25,14767.75,-207.0,-0.002342,2021-02-23 09:35:00,2021-02-23 10:15:00,0 days 00:40:00,64,2021,2,Tuesday,2021-02-23
158,6,16274,16352,14789.80,15106.10,1897.8,0.021386,2021-02-24 09:35:00,2021-02-25 15:10:00,1 days 05:35:00,65,2021,2,Wednesday,2021-02-24


In [517]:
df_trades[df_trades.flag_addup == 96]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,flag_addup,year,month,day_name
248,-5,26582,26583,17261.20,17286.40,-126.00,-0.001460,2021-09-03 09:40:00,2021-09-03 09:45:00,0 days 00:05:00,96,2021,9,Friday
249,5,26653,26662,17416.70,17379.55,-185.75,-0.002133,2021-09-06 09:35:00,2021-09-06 10:20:00,0 days 00:45:00,96,2021,9,Monday
250,-5,26869,26878,17315.80,17342.70,-134.50,-0.001553,2021-09-09 09:35:00,2021-09-09 10:20:00,0 days 00:45:00,96,2021,9,Thursday
251,-5,26947,26976,17286.40,17335.55,-245.75,-0.002843,2021-09-13 10:05:00,2021-09-13 12:30:00,0 days 02:25:00,96,2021,9,Monday
252,5,27013,27016,17435.80,17409.55,-131.25,-0.001506,2021-09-14 09:35:00,2021-09-14 09:50:00,0 days 00:15:00,96,2021,9,Tuesday
253,5,27157,27160,17563.85,17548.55,-76.50,-0.000871,2021-09-16 09:35:00,2021-09-16 09:50:00,0 days 00:15:00,96,2021,9,Thursday
254,-5,27230,27231,17714.00,17735.15,-105.75,-0.001194,2021-09-17 09:40:00,2021-09-17 09:45:00,0 days 00:05:00,96,2021,9,Friday
255,5,27301,27356,17536.30,17462.05,-371.25,-0.004234,2021-09-20 09:35:00,2021-09-20 14:10:00,0 days 04:35:00,96,2021,9,Monday
256,5,27375,27378,17463.80,17410.35,-267.25,-0.003061,2021-09-21 09:45:00,2021-09-21 10:00:00,0 days 00:15:00,96,2021,9,Tuesday
257,5,27449,27468,17579.90,17534.80,-225.50,-0.002565,2021-09-22 09:55:00,2021-09-22 11:30:00,0 days 01:35:00,96,2021,9,Wednesday


In [514]:
df_trades['year']  = df_trades['EntryTime'].dt.year
df_trades['month']  = df_trades['EntryTime'].dt.month
df_trades['day_name']  = df_trades['EntryTime'].apply(lambda x: datetime.strftime(x, '%A'))
pd.pivot_table(df_trades, index = "year", columns = ['day_name'], values = "PnL",aggfunc= sum)

day_name,Friday,Monday,Thursday,Tuesday,Wednesday
year,,,,,
2020,-2239.10,351.15,1310.95,-2989.9,2408.15
2021,1931.45,3445.50,2787.25,-2949.4,-1267.20
2022,-2605.50,7795.85,5022.20,807.5,3689.80
2023,NaN,1189.50,-663.30,1253.4,1498.50


## fibonacci

In [ ]:
# 1. calculate the diffrence of high and low of first candle
# 2. multiply the diffrence with [0.312,.618,1.618,2.618]
# 3. ADD the above value to low_high and subtract  lower value from low
# 4. buy and sell  when level breaches  .618 either upside or downside
# 5. target 1 is 1.618
# 6. sl is .312( very tight), high or low either side

In [306]:
start_dt = '2019-01-01'
end_dt = '2023-02-02'
time_frame = '5minute'
instument =256265 #256265 # 260105(bank) #341249(hdfc)
df_5min = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
df_day_fetch = uf.get_data_parllel(kite, instument, 'day' , start_dt,end_dt)
temp1 = df_day_fetch.copy()
temp2 = df_5min.copy()

Pandas Apply:   0%|          | 0/49 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/49 [00:00<?, ?it/s]

In [307]:
df_day_fetch = temp1.copy()
df_5min = temp2.copy()

In [308]:
df_5min['first_candle_diff'] = np.where(df_5min.index.time == pd.to_datetime('09:15:00').time(),df_5min['High'] - df_5min['Low'] , np.nan)
df_5min['first_candle_high'] = np.where(df_5min.index.time == pd.to_datetime('09:15:00').time(),df_5min['High'] , np.nan)
df_5min['first_candle_low'] = np.where(df_5min.index.time == pd.to_datetime('09:15:00').time(),df_5min['Low'] , np.nan)
df_5min.ffill(inplace=True)
df_5min = df_5min.between_time('09:15', '15:20').copy()
df_5min.index = df_5min.index.tz_localize(None)
# df_5min.dropna(inplace = True)

In [ ]:
def low_high(data):
    # print(data)
    return data

def fib_indicator(high,first_candle, target):
    return (high + (first_candle* target))

def fib_indicator_below(low,first_candle, target):
    return (low  - (first_candle* target))  
    
class fibonacci_first(Strategy): 
    flag = 0
    date_list = []

    break_level = 0.618
    target_level = 1.618
    start_candle  = '09:15:00'

    def init(self):
        # indicator of buy
        self.break_level_above = self.I(fib_indicator, self.data.first_candle_high,self.data.first_candle_diff,self.break_level)
        self.target_level_above = self.I(fib_indicator, self.data.first_candle_high,self.data.first_candle_diff,self.target_level)
        self.sl_level_above = self.I(low_high,self.data.first_candle_low, plot = False)
        # indicator for sell
        self.break_level_below = self.I(fib_indicator_below, self.data.first_candle_low,self.data.first_candle_diff,self.break_level)
        self.target_level_below = self.I(fib_indicator_below, self.data.first_candle_low,self.data.first_candle_diff,self.target_level)
        self.sl_level_below = self.I(low_high,self.data.first_candle_high, plot = False)

        
    def next(self):
        # getting 1 trade a day
        if (self.data.index.date[-1] not in self.date_list) and (self.data.index[-1].time() == pd.to_datetime(self.start_candle).time()):
            self.flag = 3
            self.date_list.append(self.data.index.date[-1])

        if self.data.index[-1].time() == pd.to_datetime('15:20:00').time():
            self.position.close()
        try:
            # print(self.sl_level_below,self.break_level_below,self.target_level_below)
            if self.break_level_above[-1] < self.data.Close[-1] and self.data.index[-1].time() > pd.to_datetime(self.start_candle).time() and not self.position and self.flag > 0:
                self.buy(sl = self.sl_level_above[-1], tp = self.target_level_above[-1])
                self.flag -= 1
                
            if self.break_level_below[-1] > self.data.Close[-1] and \
                self.data.index[-1].time() > pd.to_datetime(self.start_candle).time() and self.flag > 0 \
                and not self.position:
                # print(self.data)
                self.sell(sl = self.sl_level_below[-1], tp = self.target_level_below[-1])
                self.flag -= 1
        except:
            # print("try",self.data)
            # print('try')
            pass

bt = Backtest(df_5min,fibonacci_first, cash = 100000)

In [314]:
stats = bt.run()
stats

In [255]:
df_trades = stats._trades
df_trades['entry_date'] = df_trades['EntryTime'].dt.date
df_trades['entry_date'].value_counts()

2022-02-28    3
2022-01-20    3
2023-01-06    3
2022-01-05    2
2022-06-10    2
             ..
2022-05-25    1
2022-05-13    1
2022-05-11    1
2022-05-05    1
2023-01-20    1
Name: entry_date, Length: 98, dtype: int64

In [256]:
df_trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,entry_date
0,5,14,18,17820.10,17931.4484,556.7420,0.006248,2022-01-05 09:15:00,2022-01-05 13:15:00,0 days 04:00:00,2022-01-05
1,5,20,21,17935.40,17768.5000,-834.5000,-0.009306,2022-01-05 15:15:00,2022-01-06 09:15:00,0 days 18:00:00,2022-01-05
2,5,55,56,18215.10,18257.0000,209.5000,0.002300,2022-01-12 15:15:00,2022-01-13 09:15:00,0 days 18:00:00,2022-01-12
3,-5,83,84,18096.55,18129.2000,-163.2500,-0.001804,2022-01-18 15:15:00,2022-01-19 09:15:00,0 days 18:00:00,2022-01-18
4,-5,95,95,17769.80,17680.7467,445.2665,0.005011,2022-01-20 13:15:00,2022-01-20 13:15:00,0 days 00:00:00,2022-01-20
...,...,...,...,...,...,...,...,...,...,...,...
116,-6,1761,1761,17831.70,17803.9114,166.7316,0.001558,2023-01-06 13:15:00,2023-01-06 13:15:00,0 days 00:00:00,2023-01-06
117,-6,1762,1762,17820.60,17803.9114,100.1316,0.000936,2023-01-06 14:15:00,2023-01-06 14:15:00,0 days 00:00:00,2023-01-06
118,-6,1763,1764,17869.45,17952.5500,-498.6000,-0.004650,2023-01-06 15:15:00,2023-01-09 09:15:00,2 days 18:00:00,2023-01-06
119,6,1796,1799,17964.35,18033.1500,412.8000,0.003830,2023-01-13 13:15:00,2023-01-16 09:15:00,2 days 20:00:00,2023-01-13


In [257]:
bt.plot(resample=False)

c:\Users\akash.verma\Miniconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  """Entry point for launching an IPython kernel.


Row(id='19871', ...)

In [103]:
df_trades['days'] = df_trades.Duration.dt.days
df_trades[df_trades['days']> 0]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,entry_date,days
11,5,728,740,17967.8,18033.15,326.75,0.003637,2023-01-13 14:25:00,2023-01-16 09:15:00,2 days 18:50:00,2023-01-13,2


In [799]:

df_trades['year']  = df_trades['EntryTime'].dt.year
df_trades['month']  = df_trades['EntryTime'].dt.month
df_trades['day_name']  = df_trades['EntryTime'].apply(lambda x: datetime.strftime(x, '%A'))
pd.pivot_table(df_trades, index = "year", columns = ['month'], values = "PnL",aggfunc= sum)

month,1,2,3,4,5,6,7,8,9,10,11,12
year,,,,,,,,,,,,
2022,1516.4200,483.4158,6110.3268,-1045.8624,-215.1204,1846.3574,2738.1384,2905.4556,3710.0412,2112.5454,-17.9346,1276.0614
2023,2005.4628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## MACD

In [188]:
import ta

In [210]:
start_dt = '2023-01-01'
end_dt = '2023-02-02'
time_frame = '60minute'
instument =256265 #256265 # 260105(bank) #341249(hdfc)
df_5min = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
df_day_fetch = uf.get_data_parllel(kite, instument, 'day' , start_dt,end_dt)
temp1 = df_day_fetch.copy()
temp2 = df_5min.copy()

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

In [211]:
df_day_fetch = temp1.copy()
df_5min = temp2.copy()

In [212]:
df_5min = df_5min.between_time('09:15', '15:20').copy()
df_5min.index = df_5min.index.tz_localize(None)

In [213]:

    
class Mcd_strategy(Strategy): 
    flag = 0
    date_list = []
    start_candle  = '09:15:00'

    def init(self):
        # indicator of buy
        close = self.data.Close
        self.macd = self.I(ta.trend.macd, pd.Series(close))
        self.macd_signal = self.I(ta.trend.macd_signal, pd.Series(close))
        self.ema_100 = self.I(ta.trend.ema_indicator, pd.Series(close), window = 100)

    def next(self):
        price = self.data.Close  

        if (self.data.index.date[-1] not in self.date_list) and (self.data.index[-1].time() == pd.to_datetime(self.start_candle).time()):
            self.flag = 3
            self.date_list.append(self.data.index.date[-1])
        
        elif self.data.index[-1].time() == pd.to_datetime('15:20:00').time():
            self.position.close()
         
        elif crossover(self.macd, self.macd_signal) and (price > self.ema_100) and not self.position and self.flag > 0:
            sl = price * .97
            tp = price * 1.045
            self.buy(sl = sl , tp = tp)
            self.flag -= 1
        # getting 1 trade a day


        # try:
        #     # print(self.sl_level_below,self.break_level_below,self.target_level_below)
        #     if self.break_level_above[-1] < self.data.Close[-1] and self.data.index[-1].time() > pd.to_datetime(self.start_candle).time() and not self.position and self.flag > 0:
        #         self.buy(sl = self.sl_level_above[-1], tp = self.target_level_above[-1])
        #         self.flag -= 1
                
        #     if self.break_level_below[-1] > self.data.Close[-1] and \
        #         self.data.index[-1].time() > pd.to_datetime(self.start_candle).time() and self.flag > 0 \
        #         and not self.position:
        #         # print(self.data)
        #         self.sell(sl = self.sl_level_below[-1], tp = self.target_level_below[-1])
        #         self.flag -= 1
        # except:
        #     # print("try",self.data)
        #     # print('try')
        #     pass

bt = Backtest(df_5min,Mcd_strategy, cash = 100000)

In [214]:
stats = bt.run()
stats

Start                     2023-01-02 09:15:00
End                       2023-01-20 15:15:00
Duration                     18 days 06:00:00
Exposure Time [%]                         0.0
Equity Final [$]                     100000.0
Equity Peak [$]                      100000.0
Return [%]                                0.0
Buy & Hold Return [%]               -0.746577
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [209]:
bt.plot()

Row(id='18559', ...)

## vector by

In [9]:
import vectorbt as vbt

In [11]:
start_dt = '2023-01-01'
end_dt = '2023-02-02'
time_frame = '5minute'
instument =256265 #256265 # 260105(bank) #341249(hdfc)
df_5min = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
df_day_fetch = uf.get_data_parllel(kite, instument, 'day' , start_dt,end_dt)
temp1 = df_day_fetch.copy()
temp2 = df_5min.copy()

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
fast_ma = vbt.MA.run(df_5min['Close'], 10, short_name='fast')
slow_ma = vbt.MA.run(df_5min['Close'], 20, short_name='slow')
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

In [25]:
fast_ma

AttributeError: 'MA' object has no attribute 'count'

In [21]:
pf = vbt.Portfolio.from_signals(df_5min['Close'], entries, exits)

In [22]:
pf.total_return()

0.004697263527415459

## bolinger band

In [ ]:
# bolinger band 
# 1.5
# time frame = 15 min
# ATM  or slightly out the money
# sl = 100 point maximum
# target 1:4 
# if first candle price is outside bolinger band on up or Down
# buy or sell -> 1:5

In [28]:
# f_high = 42894
# f_low = 42745
# print(f_high * .618,f_high * 1.618)
# print(f_low * .618,f_low * 1.618)

In [111]:
start_dt = '2022-01-01'
end_dt = '2023-02-02'
time_frame = '15minute'
instument =260105 #256265(nifty) # 260105(bank) #341249(hdfc)
df_5min = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
# df_day_fetch = uf.get_data_parllel(kite, instument, 'day' , start_dt,end_dt)
# temp1 = df_day_fetch.copy()
temp2 = df_5min.copy()

Pandas Apply:   0%|          | 0/13 [00:00<?, ?it/s]

In [112]:
df_5min = temp2.copy()
df_5min = df_5min.between_time('09:15', '15:20').copy()
df_5min.index = df_5min.index.tz_localize(None)

In [116]:
class bolinger_strategy(Strategy): 
    flag = 0
    date_list = []
    std_dev = 1.5
    signal_candle_low = 0
    signal_candle_high = 0
    start_candle  = '09:15:00'
    risk_reward_ratio = .98

    def init(self):
        self.upper, self.mid, self.lower = self.I(talib.BBANDS,self.data.Close, nbdevup=self.std_dev,nbdevdn=self.std_dev,timeperiod=20)

    def next(self):  
        if (self.data.index.date[-1] not in self.date_list) and (self.data.index[-1].time() == pd.to_datetime(self.start_candle).time()):
            self.flag = 3
            self.signal_candle_high = 0
            self.signal_candle_low = 0
            self.date_list.append(self.data.index.date[-1])

        elif self.data.index[-1].time() > pd.to_datetime('15:15:00').time():
            self.position.close()

        elif self.data.Low[-1] > self.upper[-1]:
            self.signal_candle_high = self.data.High[-1]
            self.signal_candle_low = self.data.Low[-1]

        elif self.data.Close[-1] < self.signal_candle_low and  self.signal_candle_low != 0 and  self.signal_candle_high != 0 and self.flag > 0:
            self.sell(sl = self.signal_candle_high, tp = self.data.Close[-1] * self.risk_reward_ratio)
            self.flag -= 1
            self.signal_candle_high = 0
            self.signal_candle_low = 0


        # if (self.data.index.date[-1] not in self.date_list) and (self.data.index[-1].time() == pd.to_datetime(self.start_candle).time()):
        #     self.flag = 3
        #     self.date_list.append(self.data.index.date[-1])

        # if self.data.index[-1].time() == pd.to_datetime('15:20:00').time():
        #     self.position.close()
        # try:
        #     # print(self.sl_level_below,self.break_level_below,self.target_level_below)
        #     if self.break_level_above[-1] < self.data.Close[-1] and self.data.index[-1].time() > pd.to_datetime(self.start_candle).time() and not self.position and self.flag > 0:
        #         self.buy(sl = self.sl_level_above[-1], tp = self.target_level_above[-1])
        #         self.flag -= 1
                
        #     if self.break_level_below[-1] > self.data.Close[-1] and \
        #         self.data.index[-1].time() > pd.to_datetime(self.start_candle).time() and self.flag > 0 \
        #         and not self.position:
        #         # print(self.data)
        #         self.sell(sl = self.sl_level_below[-1], tp = self.target_level_below[-1])
        #         self.flag -= 1
        # except:
        #     # print("try",self.data)
        #     # print('try')
        #     pass

bt = Backtest(df_5min,bolinger_strategy, cash = 100000)

In [117]:
stats = bt.run(
    # std_dev = [1.3,1.4,1.5,1.7,1.9,2],
    # risk_reward_ratio = [.99,.98,.97,.96,.95,.94]
)
stats

Start                     2022-01-03 09:15:00
End                       2023-01-23 14:45:00
Duration                    385 days 05:30:00
Exposure Time [%]                   28.138792
Equity Final [$]                     101332.7
Equity Peak [$]                    112357.812
Return [%]                             1.3327
Buy & Hold Return [%]                20.29151
Return (Ann.) [%]                    1.276605
Volatility (Ann.) [%]                7.736312
Sharpe Ratio                         0.165015
Sortino Ratio                        0.287164
Calmar Ratio                         0.128441
Max. Drawdown [%]                   -9.939239
Avg. Drawdown [%]                   -1.006158
Max. Drawdown Duration      206 days 04:45:00
Avg. Drawdown Duration       12 days 00:36:00
# Trades                                  106
Win Rate [%]                         15.09434
Best Trade [%]                       3.885446
Worst Trade [%]                     -1.057217
Avg. Trade [%]                    

In [118]:
bt.plot()
bt._strategy

__main__.bolinger_strategy

In [119]:
df_trades = stats._trades
df_trades['entry_date'] = df_trades['EntryTime'].dt.date
df_trades['entry_date'].value_counts()

2022-04-19    2
2022-07-07    2
2022-08-18    2
2022-09-13    2
2022-09-08    2
             ..
2022-05-17    1
2022-05-10    1
2022-04-29    1
2022-04-28    1
2023-01-23    1
Name: entry_date, Length: 101, dtype: int64

In [120]:
df_trades['days'] = df_trades.Duration.dt.days
df_trades[df_trades['days']> 0]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,entry_date,days
2,-2,106,125,38031.10,38134.500,-206.800,-0.002719,2022-01-07 10:45:00,2022-01-10 09:15:00,2 days 22:30:00,2022-01-07,2
4,-2,184,291,38725.15,38848.800,-247.300,-0.003193,2022-01-12 11:30:00,2022-01-18 13:15:00,6 days 01:45:00,2022-01-12,6
12,-2,842,859,37714.15,37816.050,-203.800,-0.002702,2022-02-18 13:30:00,2022-02-21 11:30:00,2 days 22:00:00,2022-02-18,2
18,-2,1297,1298,36409.40,36545.850,-272.900,-0.003748,2022-03-17 15:15:00,2022-03-21 09:15:00,3 days 18:00:00,2022-03-17,3
24,-2,1638,1723,37878.65,37962.050,-166.800,-0.002202,2022-04-07 13:00:00,2022-04-13 09:15:00,5 days 20:15:00,2022-04-07,5
30,-2,1986,1998,36595.85,35627.600,1936.500,0.026458,2022-04-29 12:30:00,2022-05-02 09:15:00,2 days 20:45:00,2022-04-29,2
35,-3,2368,2398,34379.85,34643.100,-789.750,-0.007657,2022-05-23 14:15:00,2022-05-25 09:15:00,1 days 19:00:00,2022-05-23,1
39,-2,2485,2595,35950.20,35230.657,1439.086,0.020015,2022-05-30 12:15:00,2022-06-03 14:45:00,4 days 02:30:00,2022-05-30,4
40,-3,2663,2711,35189.90,34486.837,2109.189,0.019979,2022-06-08 13:00:00,2022-06-10 12:30:00,1 days 23:30:00,2022-06-08,1
41,-3,2759,2798,33505.50,33648.800,-429.900,-0.004277,2022-06-14 12:00:00,2022-06-16 09:15:00,1 days 21:15:00,2022-06-14,1


## END

## Rocket scanner

In [132]:
import numpy as np

In [183]:
inst[inst.tradingsymbol == 'BANKA']

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
2378,210433,822,BANKA,BANKA BIOLOO,0.0,None,0.0,0.05,1,EQ,NSE,NSE


In [184]:
start_dt = '2023-01-01'
end_dt = '2023-02-02'
time_frame = '15minute'
instument =2647809 #256265(nifty) # 260105(bank) #341249(hdfc)
# df_5min = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
df_day_fetch = uf.get_data_parllel(kite, instument, 'day' , start_dt,end_dt)
# temp1 = df_day_fetch.copy()
temp2 = df_5min.copy()

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

In [185]:
# MANAKSIA
# 1. Latest Max ( 5 , Latest Close ) > 6 days ago Max ( 120 , Latest Close ) * 1.05 
# 2. Latest Volume > Latest Sma ( volume,5 ) 
# 3. Latest Close > 1 day ago Close 
# 4. Latest Close > 50 

In [186]:
df_day_fetch['6_days_close'] = df_day_fetch['Close'].shift(6)
df_day_fetch['1_days_close'] = df_day_fetch['Close'].shift(1)
df_day_fetch['five'] = 5.0
df_day_fetch['one_twenty'] = 120.0
df_day_fetch['5_emav'] = talib.SMA(df_day_fetch['Volume'],5)

In [187]:
# df_day_fetch['five']

In [188]:
df_day_fetch['cond_1'] = df_day_fetch[['five','Close']].max(axis = 1) > (df_day_fetch[['one_twenty','6_days_close']].max(axis = 1) * 1.05)

In [189]:
df_day_fetch['cond_2'] = df_day_fetch['Volume'] > df_day_fetch['5_emav']

In [190]:
df_day_fetch['cond3'] = df_day_fetch['Close'] > df_day_fetch['1_days_close']
df_day_fetch['cond4'] = df_day_fetch['Close'] > 50

In [191]:
df_day_fetch

,Open,High,Low,Close,Volume,6_days_close,1_days_close,five,one_twenty,5_emav,cond_1,cond_2,cond3,cond4
Date,,,,,,,,,,,,,,
2023-01-02 00:00:00+05:30,153.45,154.80,148.00,150.10,14610,NaN,NaN,5.0,120.0,NaN,True,False,False,True
2023-01-03 00:00:00+05:30,150.10,154.00,150.10,152.50,4384,NaN,150.10,5.0,120.0,NaN,True,False,True,True
2023-01-04 00:00:00+05:30,159.00,159.00,150.35,154.85,22163,NaN,152.50,5.0,120.0,NaN,True,False,True,True
2023-01-05 00:00:00+05:30,158.05,158.05,151.20,154.65,7088,NaN,154.85,5.0,120.0,NaN,True,False,False,True
2023-01-06 00:00:00+05:30,156.05,156.75,144.45,149.55,18242,NaN,154.65,5.0,120.0,13297.4,True,True,False,True
2023-01-09 00:00:00+05:30,151.80,155.80,150.05,154.55,8460,NaN,149.55,5.0,120.0,12067.4,True,False,True,True
2023-01-10 00:00:00+05:30,157.00,157.00,150.55,152.40,3535,150.10,154.55,5.0,120.0,11897.6,False,False,False,True
2023-01-11 00:00:00+05:30,151.05,156.90,149.10,153.70,10205,152.50,152.40,5.0,120.0,9506.0,False,True,True,True
2023-01-12 00:00:00+05:30,156.95,184.40,152.30,184.35,339387,154.85,153.70,5.0,120.0,75965.8,True,True,True,True


## 5 star RSI

In [506]:
# monthly above 60 
# weekly above 60 
# daily near 40 
# lets try on bank nifty \
# daily, hourly and 15 mins
from pandas.tseries.frequencies import to_offset

In [507]:
def find_first_monday(year, month, day):
    d = datetime(year, int(month), 6)
    offset = -d.weekday() #weekday = 0 means monday
    return d + timedelta(offset)

In [508]:
nifty_500_df = pd.read_csv('../data/ind_nifty500list.csv')
nifty_500_df = nifty_500_df.merge(inst_filter, how = 'left', on = 'Symbol')
inst_dict = dict(zip(nifty_500_df.Symbol, nifty_500_df.instrument_token))

In [513]:
def get_5_star_signal(symbol):
     df_day_fetch = pd.read_parquet(f'../data/historical/day/{symbol}/')
     logic = {'Open'  : 'first',
          'High'  : 'max',
          'Low'   : 'min',
          'Close' : 'last',
          'Volume': 'sum'}

     df_week = df_day_fetch.copy()
     df_week = df_week.resample('W').apply(logic)
     df_week.index = df_week.index - pd.tseries.frequencies.to_offset("6D")
     df_week.index = df_week.index.tz_localize(None)
     df_week = df_week.reset_index().rename(columns = {'Date':'week_date'})

     df_month = df_day_fetch.copy()
     df_month = df_month.resample('1M').apply(logic)
     df_month.index = df_month.index.to_period('M').to_timestamp()
     df_month.index = df_month.index.tz_localize(None)
     df_month  = df_month.reset_index().rename(columns = {"Date": 'year_date'})

     df_day_fetch['rsi'] = talib.RSI(df_day_fetch['Close'], timeperiod=14)
     df_week['rsi'] = talib.RSI(df_week['Close'], timeperiod=14)
     df_month['rsi'] = talib.RSI(df_month['Close'], timeperiod=14)

     df_day_fetch.index = df_day_fetch.index.tz_localize(None)
     df_day_fetch.reset_index(inplace=True)
     df_day_fetch['week_date'] = df_day_fetch.Date.dt.to_period('W').apply(lambda r: r.start_time)
     df_day_fetch.set_index('Date',inplace=True)
     df_day_fetch['year_date'] = df_day_fetch.index.to_period('M').to_timestamp()
     df_day_fetch.reset_index(inplace=True)

     df_day_fetch = (
     df_day_fetch
     .merge(df_week, how = 'left', on = 'week_date', suffixes=('_son','_father'))
     .merge(df_month, how = 'left', on = 'year_date')
     .drop(['Volume_son','Volume_father','Volume'],axis = 1)
     .dropna()
     .set_index('Date')
     .rename(columns = {"rsi": 'rsi_grand_father'})
     )
     df_day_fetch['cond_1'] = np.where((df_day_fetch['rsi_grand_father'] > 60) &  (df_day_fetch['rsi_father']> 60),1,0 )
     df_day_fetch['cond_2'] = np.where((df_day_fetch['rsi_son'] >37) &  (df_day_fetch['rsi_son']< 43),1,0 )
     df_day_fetch['buy'] =( (df_day_fetch['cond_1'] ==1) &( df_day_fetch['cond_2'] == 1))
     
     return df_day_fetch[df_day_fetch['buy']]


In [514]:
df_final = pd.DataFrame()
for sy, itoken in tqdm(inst_dict.items()):
    # print(sy,itoken)
    try:
        df = get_5_star_signal(sy)
        df['symbol'] = sy
        df_final = df_final.append(df)
    except:
        print("error",sy)
    # break


  0%|          | 0/501 [00:00<?, ?it/s]c:\Users\akash.verma\Miniconda3\envs\py37\lib\site-packages\pandas\core\arrays\datetimes.py:1146: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,
  5%|▌         | 27/501 [00:01<00:23, 20.14it/s]

error ALOKINDS


 70%|██████▉   | 350/501 [00:17<00:06, 23.12it/s]

error PATANJALI


100%|██████████| 501/501 [00:25<00:00, 19.86it/s]


In [515]:
df_final[['symbol']].sort_index().tail(10)

,symbol
Date,
2023-01-16,MAZDOCK
2023-01-17,MAZDOCK
2023-01-17,JKLAKSHMI
2023-01-18,JKLAKSHMI
2023-01-18,MAZDOCK
2023-01-19,JKLAKSHMI
2023-01-20,JKLAKSHMI
2023-01-23,SUPREMEIND
2023-01-27,KARURVYSYA


In [480]:
# start_dt = '2019-01-01'
# end_dt = '2023-02-02'
# time_frame = '5minute'
# instument =4632577 #256265(nifty) # 260105(bank) #341249(hdfc)
# # df_15min = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
# # df_hour = uf.get_data_parllel(kite, instument, '60minute' , start_dt,end_dt)
# df_day_fetch = uf.get_data_parllel(kite, instument, 'day' , start_dt,end_dt)
# temp1 = df_day_fetch.copy()
# # temp2 = df_5min.copy()

In [ ]:
df_15min['rsi'] = talib.RSI(df_15min['Close'], timeperiod=14)
df_hour['rsi'] = talib.RSI(df_hour['Close'], timeperiod=14)
df_day_fetch['rsi'] = talib.RSI(df_day_fetch['Close'], timeperiod=14)

In [ ]:
df_day_fetch['date_only'] = df_day_fetch.index.date
df_15min_merge = df_15min.merge(df_hour, how = 'left', left_index=True, right_index=True, suffixes=('_son','_father'))
df_15min_merge.ffill(inplace=True)
df_15min_merge['date_only'] = df_15min_merge.index.date
df_15min_merge.reset_index(inplace=True)
df_15min_merge = df_15min_merge.merge(df_day_fetch, how = 'left', on = 'date_only')
df_15min_merge.set_index('Date',inplace=True)
df_15min_merge.drop(['Volume_son','Volume_father','date_only','Volume'],axis = 1,inplace=True)
df_15min_merge.dropna(inplace = True)
df_15min_merge['cond_1'] = np.where((df_15min_merge['rsi'] > 60) &  (df_15min_merge['rsi_father']> 60),1,0 )
df_15min_merge['cond_2'] = np.where((df_15min_merge['rsi_son'] >37) &  (df_15min_merge['rsi_son']< 43),1,0 )
df_15min_merge['buy'] =( (df_15min_merge['cond_1'] ==1) &( df_15min_merge['cond_2'] == 1))

In [ ]:
df_15min_merge[df_15min_merge['buy'] == 1]

,Open_son,High_son,Low_son,Close_son,rsi_son,Open_father,High_father,Low_father,Close_father,rsi_father,Open,High,Low,Close,rsi,cond_1,cond_2,buy
Date,,,,,,,,,,,,,,,,,,
2022-02-01 11:30:00+05:30,38484.55,38488.80,38424.05,38443.55,39.617699,38681.45,38743.20,38238.65,38605.00,64.447288,38460.45,38802.7,37690.60,38505.5,67.214930,1,1,True
2022-02-01 11:30:00+05:30,38484.55,38488.80,38424.05,38443.55,39.617699,38681.45,38743.20,38238.65,38605.00,64.447288,38460.45,38802.7,37690.60,38505.5,67.214930,1,1,True
2022-02-01 11:40:00+05:30,38387.85,38430.90,38341.20,38410.80,38.426735,38681.45,38743.20,38238.65,38605.00,64.447288,38460.45,38802.7,37690.60,38505.5,67.214930,1,1,True
2022-02-01 11:40:00+05:30,38387.85,38430.90,38341.20,38410.80,38.426735,38681.45,38743.20,38238.65,38605.00,64.447288,38460.45,38802.7,37690.60,38505.5,67.214930,1,1,True
2022-02-02 14:05:00+05:30,39172.80,39183.25,39090.35,39102.45,40.208641,39166.30,39273.00,39082.95,39255.25,66.354771,38841.55,39386.6,38752.65,39330.5,72.714933,1,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-14 14:10:00+05:30,44084.70,44091.85,44060.10,44060.10,42.513529,44105.05,44114.35,44042.15,44060.10,65.731053,44078.60,44151.8,43987.95,44049.1,76.742727,1,1,True
2022-12-14 14:30:00+05:30,44063.15,44084.15,44040.25,44040.25,39.694456,44063.00,44103.75,44000.30,44050.05,64.897436,44078.60,44151.8,43987.95,44049.1,76.742727,1,1,True
2022-12-14 14:45:00+05:30,44017.40,44041.30,44012.45,44026.90,39.525617,44063.00,44103.75,44000.30,44050.05,64.897436,44078.60,44151.8,43987.95,44049.1,76.742727,1,1,True


## first candle setup

In [36]:
start_dt = '2023-01-01'
end_dt = '2023-03-01'
time_frame = '5minute'
instument =260105 #256265(nifty) # 260105(bank) #341249(hdfc)
df_5min = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
# df_day_fetch = uf.get_data_parllel(kite, instument, 'day' , start_dt,end_dt)
# temp1 = df_day_fetch.copy()
temp2 = df_5min.copy()

Pandas Apply:   0%|          | 0/2 [00:00<?, ?it/s]

In [37]:
df_5min = temp2.copy()
df_5min = df_5min.between_time('09:15', '15:20').copy()
df_5min.index = df_5min.index.tz_localize(None)

In [39]:
# df_5min

In [40]:
df_5min['first_candle_high'] = np.where(df_5min.index.time == pd.to_datetime('09:15:00').time(),df_5min['High'] , np.nan)
df_5min['first_candle_low'] = np.where(df_5min.index.time == pd.to_datetime('09:15:00').time(),df_5min['Low'] , np.nan)
df_5min.ffill(inplace=True)

In [41]:
## if lower or upper candle breaks of 9:15 candle 
#then go long or short 

In [54]:

class First_candle_stretegy(Strategy): 
    flag = 0
    date_list = []
    start_candle  = '09:15:00'
    end_candle = '10:00:00'

    def init(self):
        # indicator of buy
        close = self.data.Close


    def next(self):
        price = self.data.Close  

        if (self.data.index.date[-1] not in self.date_list) and (self.data.index[-1].time() == pd.to_datetime(self.start_candle).time()):
            self.flag = 1
            self.date_list.append(self.data.index.date[-1])
             
        
        elif self.data.index[-1].time() == pd.to_datetime('15:20:00').time():
            self.position.close()

        # elif self.data.Close[-1] > self.data.first_candle_high and not self.position and (self.data.index[-1].time()  <= pd.to_datetime(self.end_candle).time()):
        #     if self.data.first_candle_high[-1] > self.data.first_candle_low[-1]:
        #         tp = self.data.Close[-1] + (self.data.first_candle_high[-1] - self.data.first_candle_low[-1])
        #         sl = min(self.data.first_candle_low[-1], self.data.Close[-1] - 70)
        #         self.buy(sl = sl, tp = tp )

        elif self.data.Close[-1] < self.data.first_candle_low[-1] and not self.position and (self.data.index[-1].time()  <= pd.to_datetime(self.end_candle).time()):
                tp = self.data.Close[-1] - ((self.data.first_candle_high[-1] - self.data.first_candle_low[-1]) * 2)
                sl = min(self.data.first_candle_high[-1], self.data.Close[-1] + 70)
                self.sell(sl = sl, tp = tp )
        
         
        # getting 1 trade a day


        # try:
        #     # print(self.sl_level_below,self.break_level_below,self.target_level_below)
        #     if self.break_level_above[-1] < self.data.Close[-1] and self.data.index[-1].time() > pd.to_datetime(self.start_candle).time() and not self.position and self.flag > 0:
        #         self.buy(sl = self.sl_level_above[-1], tp = self.target_level_above[-1])
        #         self.flag -= 1
                
        #     if self.break_level_below[-1] > self.data.Close[-1] and \
        #         self.data.index[-1].time() > pd.to_datetime(self.start_candle).time() and self.flag > 0 \
        #         and not self.position:
        #         # print(self.data)
        #         self.sell(sl = self.sl_level_below[-1], tp = self.target_level_below[-1])
        #         self.flag -= 1
        # except:
        #     # print("try",self.data)
        #     # print('try')
        #     pass

bt = Backtest(df_5min,First_candle_stretegy, cash = 100000)

In [55]:
stats = bt.run(
    # std_dev = [1.3,1.4,1.5,1.7,1.9,2],
    # risk_reward_ratio = [.99,.98,.97,.96,.95,.94]
)
stats

Start                     2023-01-02 09:15:00
End                       2023-02-13 15:20:00
Duration                     42 days 06:05:00
Exposure Time [%]                    8.153153
Equity Final [$]                     103116.9
Equity Peak [$]                      103116.9
Return [%]                             3.1169
Buy & Hold Return [%]               -4.085982
Return (Ann.) [%]                   30.878928
Volatility (Ann.) [%]                6.351533
Sharpe Ratio                          4.86165
Sortino Ratio                       30.072009
Calmar Ratio                        58.123521
Max. Drawdown [%]                   -0.531264
Avg. Drawdown [%]                   -0.138477
Max. Drawdown Duration       14 days 00:15:00
Avg. Drawdown Duration        1 days 03:00:00
# Trades                                   17
Win Rate [%]                        47.058824
Best Trade [%]                       1.131258
Worst Trade [%]                     -0.178629
Avg. Trade [%]                    

In [56]:
bt.plot()

Row(id='6507', ...)

## 4 ema , 8,13,21,55

In [20]:
start_dt = '2020-01-01'
end_dt = '2023-04-02'
time_frame = '5minute'
instument =256265 #256265 # 260105(bank) #341249(hdfc)
df_5min = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
df_day_fetch = uf.get_data_parllel(kite, instument, 'day' , start_dt,end_dt)
temp1 = df_day_fetch.copy()
temp2 = df_5min.copy()

Pandas Apply:   0%|          | 0/39 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/39 [00:00<?, ?it/s]

In [22]:
df_day_fetch.index = df_day_fetch.index.tz_localize(None)

In [25]:
from backtesting.lib import crossover

class four_ema(Strategy):
    first_ma = 8
    second_ma = 13
    third_ma  = 21
    fourth_ma = 55
    rtr = 4
    target_ = 1
    def init(self):
        self.ema_8 = self.I(talib.EMA, self.data.Close, self.first_ma)
        self.ema_13 = self.I(talib.EMA, self.data.Close, self.second_ma)
        self.ema_21= self.I(talib.EMA, self.data.Close, self.third_ma)
        self.ema_55 = self.I(talib.EMA, self.data.Close, self.fourth_ma)


    
    def next(self):
        # print(self.ema_5)
        # print(self.data.Close)
        if crossover(self.ema_55,self.ema_21):
            target = self.data.Close[-1] + (self.data.Close[-1] * .04)
            stop_loss = self.data.Close[-1] - (self.data.Close[-1] * .01)
            self.buy(sl =stop_loss, tp = target)

    


bt = Backtest(df_day_fetch,four_ema, cash = 100000)

In [31]:
stats = bt.run()
stats

Start                     2020-01-01 00:00:00
End                       2023-03-10 00:00:00
Duration                   1164 days 00:00:00
Exposure Time [%]                    4.131227
Equity Final [$]                    99385.321
Equity Peak [$]                      105223.9
Return [%]                          -0.614679
Buy & Hold Return [%]               42.933716
Return (Ann.) [%]                   -0.195007
Volatility (Ann.) [%]                2.811159
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -5.548719
Avg. Drawdown [%]                   -3.142882
Max. Drawdown Duration      463 days 00:00:00
Avg. Drawdown Duration      161 days 00:00:00
# Trades                                    5
Win Rate [%]                             20.0
Best Trade [%]                       3.956174
Worst Trade [%]                     -1.692219
Avg. Trade [%]                    

In [27]:
bt.plot()

Row(id='1491', ...)

In [32]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,6,339,353,14493.80,15067.200,3440.400,0.039562,2021-04-27,2021-05-18,21 days
1,6,494,497,17104.40,16995.231,-655.014,-0.006383,2021-12-01,2021-12-06,5 days
2,5,544,547,17370.09,17076.150,-1469.700,-0.016922,2022-02-09,2022-02-14,5 days
3,5,600,600,17096.60,16898.409,-990.955,-0.011592,2022-05-04,2022-05-04,0 days
4,5,782,791,17867.50,17679.618,-939.410,-0.010515,2023-01-13,2023-01-27,14 days


## double candle 

In [207]:
start_dt = '2023-01-01'
end_dt = '2023-04-01'
time_frame = '5minute'
instument = 260105 #256265(nifty) 260105
df_5min = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
df_day_fetch = uf.get_data_parllel(kite, instument, 'day' , start_dt,end_dt)
temp1 = df_day_fetch.copy()
temp2 = df_5min.copy()

Pandas Apply:   0%|          | 0/3 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/3 [00:00<?, ?it/s]

In [208]:
df_5min = temp2.copy()
df_5min['abs_o_c'] = abs(df_5min['Close'] - df_5min['Open'])
df_5min['O_C'] = (df_5min['Close'] - df_5min['Open'])
df_5min['up_down'] = np.where(df_5min['O_C'] > 0, 'up','down')
df_5min['mean_5'] = df_5min['abs_o_c'].rolling(3).mean()
df_5min = df_5min.between_time('09:30', '15:00').copy()
df_5min.index = df_5min.index.tz_localize(None)

In [209]:
df_5min.dropna(inplace=True)

In [210]:
df_5min['bigger_candle'] = df_5min['abs_o_c'] > (df_5min['mean_5'] * 2.5)

In [211]:
df_5min['signal_candle'] = df_5min['bigger_candle'].shift(1)

In [212]:
df_5min.dropna(inplace=True)

In [213]:
df_5min['up_down_signal'] = df_5min['up_down'].shift(1)
df_5min.dropna(inplace=True)

In [214]:
df_5min['pl'] = np.where((df_5min['signal_candle']) & 
                         ( df_5min['up_down'] == df_5min['up_down_signal']),
                         df_5min['abs_o_c'],
                         np.nan            
                         )

In [215]:
df_5min['pl'] = np.where((df_5min['signal_candle']) & 
                         ( df_5min['up_down'] != df_5min['up_down_signal']),
                         -df_5min['abs_o_c'],
                            df_5min['pl']       
                         )

In [216]:
df_5min['date_only'] = pd.to_datetime(df_5min.index.date)

In [217]:
df_5min[(df_5min['signal_candle'])].groupby('date_only')['pl'].sum()

date_only
2023-01-02      5.65
2023-01-04     37.80
2023-01-05     96.70
2023-01-06     90.65
2023-01-10    -51.85
2023-01-11   -296.25
2023-01-12     25.85
2023-01-13    -26.95
2023-01-16    -29.20
2023-01-18     -0.15
2023-01-19    -45.05
2023-01-20     40.90
2023-01-23    -85.40
2023-01-24     -0.35
2023-01-25     33.00
2023-01-27     48.10
2023-01-31   -124.00
2023-02-02    184.10
2023-02-07    -74.60
2023-02-09    -31.20
2023-02-10     -2.80
2023-02-14      1.30
2023-02-15    -23.05
2023-02-16    -22.25
2023-02-17      5.75
2023-02-20     25.80
2023-02-21    -31.80
2023-02-22    141.50
2023-02-23    -55.65
2023-02-27     15.65
2023-02-28     24.65
2023-03-01    -85.60
2023-03-08      3.80
2023-03-09    -61.45
2023-03-14    -41.85
Name: pl, dtype: float64

In [218]:
df_5min[(df_5min['signal_candle'] == True) & (df_5min['date_only'] == '2023-02-06')]

,Open,High,Low,Close,Volume,abs_o_c,O_C,up_down,mean_5,bigger_candle,signal_candle,up_down_signal,pl,date_only
Date,,,,,,,,,,,,,,


In [219]:
df_5min.sum()

Open                                                    138732371.1
High                                                   138831499.95
Low                                                    138630232.35
Close                                                   138727894.5
Volume                                                            0
abs_o_c                                                    100364.3
O_C                                                         -4476.6
up_down           upupdownupupdowndownupdowndownupupdowndownupdo...
mean_5                                                100767.683333
bigger_candle                                                    65
signal_candle                                                    65
up_down_signal    upupupdownupupdowndownupdowndownupupdowndownup...
pl                                                          -308.25
dtype: object